In [2]:
from functools import reduce
from PIL import Image, ImageDraw
from enum import Enum



In [3]:
class CardinalDirection(Enum):
    UP = 0
    LEFT = 1
    DOWN = 2
    RIGHT = 3

class GridSquare:
    def __init__(self, position: tuple[int, int]):
        self.position = position

class Obstacle(GridSquare):
    pass

class Box(Obstacle):
    def __init__(self, x, y):
        super().__init__((x, y))

class Wall(Obstacle):
    def __init__(self, x, y):
        super().__init__((x, y))

class Robot(GridSquare):
    def __init__(self, x: int, y: int, commands: list[CardinalDirection]):
        super().__init__((x,y))
        self.commands = commands
        self.command_index = 0

    def __repr__(self):
        return f'Robot(p={self.position})'


class Grid:
    MOVEMENT = {
        CardinalDirection.UP: lambda i, j: (i - 1, j),
        CardinalDirection.DOWN: lambda i, j: (i + 1, j),
        CardinalDirection.LEFT: lambda i, j: (i, j - 1),
        CardinalDirection.RIGHT: lambda i, j: (i, j + 1),
    }
    def __init__(self, data: list[list[GridSquare]], robot: Robot):
        self.width = len(data[0])
        self.height = len(data)
        self.members: dict[tuple[int,int], list[GridSquare]] = {}
        self.data = data
        self.robot = robot
        for row in data:
            for cell in row:
                self.members[cell.position] = cell

def load_input():
    robot = None
    commands = []
    with open("../../data/day15-input.txt") as f:
        sections = f.read().split('\n\n')
        assert 2 == len(sections)

        for instruction in sections[1].strip().replace('\n',''):
            if instruction == '^':
                commands.append(CardinalDirection.UP)
            elif instruction == '<':
                commands.append(CardinalDirection.LEFT)
            elif instruction == 'v':
                commands.append(CardinalDirection.DOWN)
            elif instruction == '>':
                commands.append(CardinalDirection.RIGHT)
            else:
                raise ValueError("Unrecognized instruction: " + instruction)

        data = []
        for i,line in enumerate(sections[0].split('\n')):
            row = []
            for j, c in enumerate(line):
                if c == '#':
                    row.append(Wall(i, j))
                elif c == 'O':
                    row.append(Box(i, j))
                elif c == '@':
                    if robot is not None:
                        raise ValueError('Robot already loaded')
                    robot = Robot(i, j, commands)
                    row.append(robot)
                elif c == '.':
                    row.append(GridSquare((i,j)))
                else:
                    raise ValueError("Unhandled: " + c)
            data.append(row)
        grid = Grid(data, robot)

        return grid


In [4]:
grid = load_input()